# A B C

In [ ]:
import numpy as np
import keras
from keras import backend as k
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from google.colab import drive
import tensorflow as tf
drive.mount('/content/drive')


In [ ]:
nas_net = tf.keras.applications.nasnet.NASNetMobile()

In [ ]:
def prepare_img(file):
  img_path = "/content/drive/My Drive/Neural network/HW3/"
  img = image.load_img(img_path + file, target_size = (224, 224))
  img_array = image.img_to_array(img)
  img_array_expand_dims = np.expand_dims(img_array, axis = 0)
  tf.keras.applications.nasnet.preprocess_input(img_array_expand_dims)
  return img_array_expand_dims

In [ ]:
preprocessed_image = prepare_img('rider-10.jpg')
print(preprocessed_image.shape)
predictions = nas_net.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)
results

# D

In [ ]:
if results[0][0][2]> 0.5:
  print("this picture is of type:",results[0][0][1])
else:
  print("this picture is unknown")

# E

In [ ]:
import os
import time
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras
import keras.utils
from keras import utils as np_utils

In [ ]:
PATH =  "/content/drive/My Drive/Neural network/HW3/data/"
data_path = PATH + '/DATA1'
data_dir_list = os.listdir(data_path)

In [ ]:
img_data_list=[]

for dataset in data_dir_list:
	img_list=os.listdir(data_path+'/'+ dataset)
	print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
	for img in img_list:
		img_path = data_path + '/'+ dataset + '/'+ img 
		img = image.load_img(img_path, target_size=(224, 224))
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = tf.keras.applications.nasnet.preprocess_input(x)
		img_data_list.append(x)

In [ ]:
img_data = np.array(img_data_list)
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

In [ ]:
# Define the number of classes
num_classes = 2
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:202]=0
labels[202:404]=1

#names = ['cats','dogs']
names = ['horses','humans']
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
model = tf.keras.applications.nasnet.NASNetMobile(weights='imagenet',include_top=False)
last_layer = model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(last_layer)
# add fully-connected & dropout layers
x = Dense(512, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu',name='fc-2')(x)
x = Dropout(0.5)(x)
# a softmax layer for 2 classes
out = Dense(num_classes, activation='softmax',name='output_layer')(x)

# this is the model we will train
custom_NAS_net_mode = Model(inputs=model.input, outputs=out)

custom_NAS_net_mode.summary()

for layer in custom_NAS_net_mode.layers[:-6]:
	layer.trainable = False

custom_NAS_net_mode.layers[-1].trainable

custom_NAS_net_mode.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
t=time.time()
#trained_model = custom_NAS_net_mode.fit(X_train, y_train,batch_size=64,epochs=20,validation_split=0.2)
trained_model = custom_NAS_net_mode.fit(X_train, y_train, batch_size=64, epochs=20, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_NAS_net_mode.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

In [ ]:
import matplotlib.pyplot as plt
def plot_details(trained_model):
    history=trained_model.history
    acc=history['accuracy']
    val_acc=history['val_accuracy']
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['acc','val_acc'])
    plt.title('accuracy')
    los = history['loss']
    val_los = history['val_loss']
    los = history['loss']
    val_los = history['val_loss']
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('loss')
    plt.plot(los)
    plt.plot(val_los)
    plt.legend(['loss','val_loss'])
    plt.title('loss')

In [ ]:
plot_details(trained_model)